In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import numpy as np
import cv2
from keras.layers import Dense, Concatenate
from keras.models import Model

import pandas as pd
import os
from tqdm import tqdm_notebook
import io

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint
import glob

In [ ]:
a3 = ['a3', 'a3_n1', 'a3_n6', 'a3_x1', 'a3_x2', 'a3_x3', 'a3_x4']
v2 = ['v2', 'v2_n1', 'v2_x1']

if os.path.isdir('predict') == False:
    os.mkdir('predict')

    for model in a3:
        os.mkdir(os.path.join('predict', model))
        
    for model in v2:
        os.mkdir(os.path.join('predict', model))

In [ ]:
clear_session()
config = tf.ConfigProto()
config.gpu_options.visible_device_list= '0'
#config.gpu_options.per_process_gpu_memory_fraction = 0.09
set_session(tf.Session(config=config))

In [ ]:
width = 171
up_width = 203
input_shape=(width, width, 3)
base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')
x = base_model.output
predictions = Dense(5270, activation='softmax', name='predictions')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
def color(rgb, color_var):
    r = 1. - color_var + 2 * color_var * np.random.random() 
    g = 1. - color_var + 2 * color_var * np.random.random() 
    b = 1. - color_var + 2 * color_var * np.random.random() 
    rgb[:,:,0] = 255*(rgb[:,:,0]/255)**b
    rgb[:,:,1] = 255*(rgb[:,:,1]/255)**g
    rgb[:,:,2] = 255*(rgb[:,:,2]/255)**r
    return np.clip(rgb, 0, 255)  

def r_crop(img, width, up_width):            
    x = np.random.randint(width, up_width+1)
    img = cv2.resize(img, (x, x))
    ws = np.random.randint(x-width+1)
    hs = np.random.randint(x-width+1)
    return img[ws:ws+width, hs:hs+width,:] 

def get_img(key, num=12, hflip_mode=1, color_var=0.2, width=171, up_width=203, nth=0, nth_e=4, rgb=True):
    n_imgs = key[1]
    nn = num//n_imgs
    
    imgs = []
    for i in range(nth, min(n_imgs, nth_e)):
        fn='test/%d-%d.jpg'%(key[0], i)
        img2 = cv2.imread(fn)            
        
        if rgb == True:
            img2 = img2[:,:,::-1]
            
        for j in range(nn):
            img = img2.copy()
            img = r_crop(img, width, up_width)

            if color_var > 0:
                img = color(img, color_var)
            
            if hflip_mode == 1:
                hflip_prob = 0.5
            elif hflip_mode == 2 and nn > 1:
                hflip_prob = j/(nn-1)
            else:
                hflip_prob = j%2
            
            if np.random.random() < hflip_prob:
                img = img[:, ::-1]
            
            imgs.append(img.astype('float32')/127.5 - 1)
    return np.array(imgs)

In [ ]:
test_imgs = pd.read_csv('test_imgs.csv').values[:,[1,2]]

def Predict_a3(sub_set = 'a3'):
    weights_file = 'weights/%s.h5'%sub_set
    model.load_weights(weights_file)
    print(weights_file)

    n = 1768182
    inn=[12,16,18,24]
    nth_e = 4
    nth   = 0
    
    if sub_set == 'a3_n1':
        nth_e = 1
        
    if sub_set == 'a3_n6':
        nth = 1
    
    for i in range(n):
        key = test_imgs[i]
        n_imgs = key[1]

        if sub_set == 'a3_x1' and n_imgs != 1 : continue
        if sub_set == 'a3_x2' and n_imgs != 2 : continue
        if sub_set == 'a3_x3' and n_imgs != 3 : continue
        if sub_set == 'a3_x4' and n_imgs != 4 : continue
        if sub_set == 'a3_n6' and n_imgs == 1 : continue

        num=inn[n_imgs-1]
        imgs = get_img(key, num=num, hflip_mode=2, color_var=0.2, width=171, up_width=203, nth=nth, nth_e=nth_e)
        pred = model.predict(imgs) 

        score = []
        n1 = num//n_imgs
        
        for n in range(min(n_imgs, nth_e)-nth):
            score.append(np.mean(pred[n1*n:n1*(n+1)], axis=0))
            
        np.save('predict/%s/%d'%(sub_set, i), score)


def Predict_v2(sub_set = 'v2'):
    weights_file = 'weights/%s.h5'%sub_set
    model.load_weights(weights_file)
    print(weights_file)

    n = 1768182
    inn=[12,16,18,24]
    nth_e = 4
    nth   = 0
    
    if sub_set == 'v2_n1':
        nth_e = 1
        
    for i in range(n):
        key = test_imgs[i]
        n_imgs = key[1]

        if sub_set == 'v2_x1' and n_imgs != 1 : continue

        num=inn[n_imgs-1]
        imgs = get_img(key, num=num, hflip_mode=2, color_var=0, width=171, up_width=203, nth=nth, nth_e=nth_e, rgb=False)
        pred = model.predict(imgs) 

        score = []
        n1 = num//n_imgs
        
        for n in range(min(n_imgs, nth_e)-nth):
            score.append(np.mean(pred[n1*n:n1*(n+1)], axis=0))
            
        np.save('predict/%s/%d'%(sub_set, i), score)


In [ ]:
for sub_set in a3:
    Predict_a3(sub_set)
    
for sub_set in v2:
    Predict_v2(sub_set)